<a href="https://colab.research.google.com/github/jun1okamura/SecurityCamp-XLS/blob/main/Jun_ichi_Okamura_xls_workshop_tohoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛠️ XLS ワーショップ

[XLS](https://google.github.io/xls/)は高位合成(High Level Synthesis: HLS)のツールチェインを提供します。XLSを用いることで、ソフトウェア開発に近い方法でハードウェアの設計が可能です。

![img](https://google.github.io/xls/images/xls_stack_diagram.png)

In [ ]:
#@title 📦 XLSのインストール {display-mode: "form"}
#@markdown - ▷ ボタンをクリックすると、XLSのセットアップが開始されます。

xls_version = 'v0.0.0-7122-gade506a92' #@param {type:"string"}
xls_colab_version = 'v0.0.0-7122-gade506a92'
rules_hdl_version = '2eb050e80a5c42ac3ffdb7e70392d86a6896dfc7' #@param {type:"string"}

!echo '📦 downloading xls-{xls_version}'
!curl --show-error -L https://github.com/proppy/xls/releases/download/{xls_version}/xls-{xls_version}-linux-x64.tar.gz | tar xzf - --strip-components=1
!echo '🧪 setting up colab integration'
import sys
!{sys.executable} -m pip install --upgrade protobuf
!{sys.executable} -m pip install --quiet --no-cache-dir --ignore-installed https://github.com/proppy/xls/releases/download/{xls_colab_version}/xls_colab-0.0.0-py3-none-any.whl
import xls.contrib.colab
_ = xls.contrib.colab.register_dslx_magic()

!echo '🧰 generating PDK metadata'
!curl --show-error -L  https://github.com/hdl/bazel_rules_hdl/archive/{rules_hdl_version}.tar.gz | tar xzf - --strip-components=1
!curl -L -O https://github.com/protocolbuffers/protobuf/releases/download/v24.3/protoc-24.3-linux-x86_64.zip
!unzip -q -o protoc-24.3-linux-x86_64.zip
import sys
!gsutil cp gs://proppy-eda/pdk_info_asap7.zip .
!unzip -q -o pdk_info_asap7.zip
!mkdir -p org_theopenroadproject_asap7sc7p5t_28/{LEF,techlef_misc} asap7/dependency_support/org_theopenroadproject_asap7_pdk_r1p7/
!cp asap7/asap7sc7p5t_28_R_1x_220121a.lef org_theopenroadproject_asap7sc7p5t_28/LEF/
!cp asap7/asap7_tech_1x_201209.lef org_theopenroadproject_asap7sc7p5t_28/techlef_misc/
!cp asap7/asap7_rvt_1x_SS.lib org_theopenroadproject_asap7sc7p5t_28/
!cp asap7/tracks.tcl asap7/dependency_support/org_theopenroadproject_asap7_pdk_r1p7/
!cp asap7/pdn_config.pdn asap7/dependency_support/org_theopenroadproject_asap7_pdk_r1p7/
!cp asap7/rc_script.tcl asap7/dependency_support/org_theopenroadproject_asap7_pdk_r1p7/
!bin/protoc --python_out=. pdk/proto/pdk_info.proto
!ln -sf pdk/proto/pdk_info_pb2.py
import pdk_info_pb2

import enum
import dataclasses
import json
import pathlib
import subprocess
from typing import Any, Callable, Dict, Optional, Union

from google.colab import widgets
from google.protobuf import text_format
import pandas as pd

yosys = 'oss-cad-suite/bin/yosys'
openroad = 'openroad'
yosys_tcl = 'synthesis/synth.tcl'

default_work_dir = xls.contrib.colab.default_work_dir

def pdk_info_proto(
    path: pathlib.Path, optional: bool = False
) -> Optional[pdk_info_pb2.PdkInfoProto]:
  """Load PDK info from prototext.

  Args:
    path: path to prototext file.
    optional: if True, failure to access the pdk info will not produce an error.

  Returns:
    Decoded pdk info proto or None if optional.
  """
  if optional and not path.exists():
    return None
  with path.open('r') as f:
    proto = pdk_info_pb2.PdkInfoProto()
    text_format.Parse(f.read(), proto)
    return proto

pdks = {
    'asap7': {
        'delay_model': 'asap7',
        'pdk_info': pdk_info_proto(
            pathlib.Path('asap7/asap7_data_pdk_info.textproto'),
        ),
    },
}

pdk = 'asap7'

@dataclasses.dataclass(frozen=True)
class RelativeCoreArea:
  utilization_percent: float


@dataclasses.dataclass(frozen=True)
class AbsoluteCoreArea:
  core_width_microns: int
  core_padding_microns: int


@enum.unique
class ImplementationStep(enum.Enum):
  """Steps in the implementation flow."""

  XLS = 'xls'
  SYNTHESIS = 'synthesis'
  PLACEMENT = 'placement'


class PdkRuntimeError(RuntimeError):
  pass


class OpenroadRuntimeError(RuntimeError):
  pass


class OpenstaRuntimeError(RuntimeError):
  pass


class YosysRuntimeError(RuntimeError):
  pass


@dataclasses.dataclass(frozen=True)
class SynthesisResults:
  synth_v: pathlib.Path
  design_stats: pd.DataFrame
  cell_stats: pd.DataFrame


def run_synthesis(
    *,
    selected_pdk: Optional[str] = None,
    work_dir: pathlib.Path = default_work_dir,
    silent: bool = False,
) -> SynthesisResults:
  """Run synthesis with Yosys.

  Args:
    selected_pdk: The pdk to use.
    work_dir: Directory that contains verilog and will be where outputs are put.
    silent: Suppress output.

  Returns:
    Metrics from running synthesis.

  Raises:
    PdkRuntimeError: on PDK error.
    YosysRuntimeError: on yosys error.
  """
  if selected_pdk is None:
    selected_pdk = pdk
  pdk_info = pdks[selected_pdk]['pdk_info']
  if pdk_info is None:
    raise PdkRuntimeError(f'PDK "{selected_pdk}" is restricted')

  liberty = (pathlib.Path(pdk) / pathlib.Path(pdk_info.liberty_path).name).resolve()
  synth_v = (work_dir / 'user_module_synth.v').resolve()
  synth_v_flist = (work_dir / 'user_module_synth_v.flist').resolve()
  synth_uhdm_flist = (work_dir / 'user_module_synth_uhdm.flist').resolve()
  synth_uhdm_flist.touch()
  synth_stats_json = (work_dir / 'user_module_synth_stats.json').resolve()
  dont_use_args = ' '.join(
      f'-dont_use {pat}'
      for pat in pdk_info.do_not_use_cell_list
  )
  # run yosys synthesis
  with synth_v_flist.open('w') as f:
    top_v = work_dir / 'user_module.sv'
    f.write(str(top_v.resolve()))
  !FLIST='{synth_v_flist}' ABC_SCRIPT='' CONSTR='' TOP='user_module' OUTPUT='{synth_v}' UHDM_FLIST='{synth_uhdm_flist}' LIBERTY='{liberty}' STATS_JSON='{synth_stats_json}' DONT_USE_ARGS='{dont_use_args}' {yosys} -c '{yosys_tcl}'
  with synth_stats_json.open('r') as f:
    synth_stats = json.load(f)
  design_stats = synth_stats['design']
  cells_stats = design_stats.pop('num_cells_by_type')
  design_stats = pd.DataFrame.from_dict(
      design_stats, orient='index', columns=['cells']
  )
  cells_stats = pd.DataFrame.from_dict(
      cells_stats, orient='index', columns=['stats']
  )

  return SynthesisResults(
      synth_v=synth_v, design_stats=design_stats, cell_stats=cells_stats
  )


def run_opensta(
    *,
    selected_pdk: Optional[str] = None,
    work_dir: pathlib.Path = default_work_dir,
    silent: bool = False,
) -> pd.DataFrame:
  """Run OpenSta and collect timing metrics.

  Args:
    selected_pdk: The pdk to use.
    work_dir: Directory that contains verilog.
    silent: Suppress output.

  Returns:
    Dataframe containing timing report.

  Raises:
    OpenstaRuntimeError: on OpenSTA error.
    PdkRuntimeError: on PDK error.
  """
  if selected_pdk is None:
    selected_pdk = pdk
  pdk_info = pdks[selected_pdk]['pdk_info']
  if pdk_info is None:
    raise PdkRuntimeError(f'PDK "{selected_pdk}" is restricted')

  liberty = pathlib.Path(pdk) / pdk_info.liberty_path
  tech_lef = pathlib.Path(pdk) / pdk_info.tech_lef_path
  read_cell_lefs = '\n'.join(
      f'read_lef {pathlib.Path(pdk) / cell_lef_path}'
      for cell_lef_path in pdk_info.cell_lef_paths
  )
  synth_v = work_dir / 'user_module_synth.v'
  top = 'user_module'
  opensta_log = work_dir / 'user_module_sta.log'

  openroad_script = f"""
  sta::redirect_file_begin {opensta_log}
  read_lef {tech_lef}
  {read_cell_lefs}
  read_liberty {liberty}
  read_verilog {synth_v}
  link_design  {top}
  report_checks -unconstrained
  sta::redirect_file_end
  """
  openroad_tcl = work_dir / 'openroad_sta.tcl'
  with openroad_tcl.open('w') as f:
    f.write(openroad_script)

  # run opensta static timing analysis
  !{openroad} {openroad_tcl} -exit

  columns = ['delay', 'time', 'edge', 'net', 'gate']

  import re
  def sta_report_paths(opensta_log):
    with open(opensta_log) as f:
      sta_report = f.read()
    m = re.search(r'---+(.*)---+', sta_report, flags=re.M | re.S)
    for path in m.group(1).split('\n')[1:-2]:
      parts = path.split(None, maxsplit=len(columns) - 1)
      yield float(parts[0]), float(parts[1]), parts[2], parts[3], parts[4]

  df = pd.DataFrame.from_records(sta_report_paths(opensta_log), columns=columns)
  df['gate'] = df['gate'].str.replace('[()]', '', regex=True)

  return df

@dataclasses.dataclass(frozen=True)
class PlacementResults:
  openroad_global_placement_layout: pathlib.Path
  metrics: pd.DataFrame
  power: pd.DataFrame

def run_placement(
    *,
    clock_period_ps: int,
    placement_density: float,
    core_area: Union[RelativeCoreArea, AbsoluteCoreArea],
    selected_pdk: Optional[str] = None,
    work_dir: pathlib.Path = default_work_dir,
    silent: bool = False,
) -> PlacementResults:
  """Run OpenRoad placement.

  Args:
    clock_period_ps: Clock period in picoseconds.
    placement_density: Placement density in [0.0, 1.0].
    core_area: Relative or absolute core area specification.
    selected_pdk: The pdk to use.
    work_dir: Directory that contains verilog and will be where outputs are put.
    silent: Suppress output.

  Returns:
    Outputs from running placement.

  Raises:
    OpenroadRuntimeError: on OpenRoad error.
    OpenstaRuntimeError: on OpenSTA error.
    PdkRuntimeError: on PDK error.
    ValueError: on invalid inputs.
    YosysRuntimeError: on yosys error.
  """
  clock_period_ns = clock_period_ps / 1000.0
  if selected_pdk is None:
    selected_pdk = pdk
  pdk_info = pdks[selected_pdk]['pdk_info']
  if pdk_info is None:
    raise PdkRuntimeError(f'PDK "{selected_pdk}" is restricted')

  liberty = pathlib.Path(pdk) / pdk_info.liberty_path
  tech_lef = pathlib.Path(pdk) / pdk_info.tech_lef_path
  read_cell_lefs = '\n'.join(
      f'read_lef {pathlib.Path(pdk) / cell_lef_path}'
      for cell_lef_path in pdk_info.cell_lef_paths
  )

  if isinstance(core_area, AbsoluteCoreArea):
    die_side_microns = (
        core_area.core_width_microns + core_area.core_padding_microns * 2
    )
    core_side_microns = (
        core_area.core_width_microns + core_area.core_padding_microns
    )
    initialize_floorplan_args = (
        f' -die_area "0 0 {die_side_microns} {die_side_microns}" -core_area'
        f' "{core_area.core_padding_microns} {core_area.core_padding_microns} {core_side_microns} {core_side_microns}"'
    )
  elif isinstance(core_area, RelativeCoreArea):
    initialize_floorplan_args = (
        f' -utilization {core_area.utilization_percent} -aspect_ratio 1.0'
    )
  else:
    raise ValueError(
        'Expected core_area to be AbsoluteCoreArea or RelativeCoreArea, got'
        f' {core_area!r}'
    )

  initialize_floorplan_command = (
      f'initialize_floorplan -site "{pdk_info.cell_site}"'
      f' {initialize_floorplan_args}'
  )

  def source_pdk_info_tcl(path):
    return f'source {pathlib.Path(pdk) / path}' if path else ''

  source_tracks_file = source_pdk_info_tcl(pdk_info.tracks_file_path)
  source_rc_script_configuration = source_pdk_info_tcl(
      pdk_info.rc_script_configuration_path
  )
  source_pdn_config = source_pdk_info_tcl(pdk_info.pdn_config_path)
  if pdk_info.tapcell_tcl_path:
    tapcell_command = source_pdk_info_tcl(pdk_info.tapcell_tcl_path)
  else:
    tapcell_command = (
        f'tapcell -distance {pdk_info.tapcell_distance} -tapcell_master'
        f' {pdk_info.tap_cell}'
    )

  synth_v = work_dir / 'user_module_synth.v'
  openroad_metrics = work_dir / 'openroad_metrics.json'
  openroad_global_placement_layout = work_dir / 'openroad_global_placement.png'

  global_routing_layer_adjustments = '\n'.join(
    f'set_global_routing_layer_adjustment {layer} {adjustement}'
    for (layer, adjustement) in pdk_info.global_routing_layer_adjustments.items()
  )
  openroad_script = f"""
  read_lef {tech_lef}
  {read_cell_lefs}
  read_liberty {liberty}
  read_verilog {synth_v}
  link_design user_module
  {initialize_floorplan_command}
  {source_tracks_file}
  insert_tiecells {pdk_info.tie_high_port} -prefix "TIE_ONE_"
  insert_tiecells {pdk_info.tie_low_port} -prefix "TIE_ZERO_"
  create_clock [get_ports clk] -period {clock_period_ns}
  {source_rc_script_configuration}
  set_wire_rc -signal -layer "{pdk_info.wire_rc_signal_metal_layer}"
  set_wire_rc -clock  -layer "{pdk_info.wire_rc_clock_metal_layer}"
  place_pins -hor_layers {pdk_info.pin_horizontal_metal_layer} -ver_layers {pdk_info.pin_vertical_metal_layer}
  {tapcell_command}
  {source_pdn_config}
  pdngen -verbose
  global_placement -timing_driven -routability_driven -density {placement_density} -pad_left {pdk_info.global_placement_cell_pad} -pad_right {pdk_info.global_placement_cell_pad}
  remove_buffers
  estimate_parasitics -placement
  repair_design
  repair_timing
  detailed_placement
  improve_placement
  optimize_mirroring
  check_placement -verbose
  {global_routing_layer_adjustments}
  set_routing_layers -signal {pdk_info.global_routing_signal_layers} -clock {pdk_info.global_routing_clock_layers}
  global_route -congestion_iterations 100
  set_propagated_clock [all_clocks]
  detailed_route
  report_power
  report_design_area
  report_wire_length -net * -detailed_route
  report_wns
  utl::metric "utilization_percent" [rsz::utilization]
  utl::metric "design_area" [rsz::design_area]
  utl::metric "power" [sta::design_power [sta::parse_corner {{}}]]
  utl::metric "wns" [sta::worst_slack -max]
  if {{[info procs save_image] == "save_image"}} {{
    save_image -resolution 0.005 "{openroad_global_placement_layout}"
  }}
  """
  openroad_tcl = work_dir / 'place.tcl'
  with openroad_tcl.open('w') as f:
    f.write(openroad_script)
  !QT_QPA_PLATFORM=minimal {openroad} -metrics {openroad_metrics} -exit {openroad_tcl}

  with open(work_dir / 'openroad_metrics.json', 'r') as f:
    metrics = json.loads(f.read())
  metrics_power = [float(m) * 1e6 for m in metrics['power'].split(' ')]
  df_power = pd.DataFrame().from_dict(
      {
          'sequential': metrics_power[4:8],
          'combinational': metrics_power[8:12],
          'clock': metrics_power[12:16],
          'macro': metrics_power[16:20],
          'pad': metrics_power[20:],
          'total': metrics_power[0:4],
      },
      orient='index',
      columns=['internal', 'switching', 'leakage', 'total'],
  )
  df_metrics = (
      pd.DataFrame.from_records([metrics])
      .transpose()
      .set_axis(['metrics'], axis=1)
  )
  return PlacementResults(
      openroad_global_placement_layout=openroad_global_placement_layout,
      metrics=df_metrics,
      power=df_power,
  )

##📜 DSLXを使った設計

[DSLX](https://google.github.io/xls/dslx_reference/) はハードウェア設計向けのデータフロー志向・関数型のドメイン特化言語(Domain Specific Language: DSL)です。DSLXで記述した高レベルな機能の設計から、具体的なハードウェアデザインを生成できます。

In [ ]:
%%dslx --top=inverter --pipeline_stages=1 --flop_inputs=false --flop_outputs=false

fn inverter(x: u1) -> u1 {
    !x
}

#[test]
fn inverter_test() {
  assert_eq(inverter(u1:0b0), u1:0b1);
  assert_eq(inverter(u1:0b1), u1:0b0);
}

## 🧪 Synthesis

### 回路合成

- ツール: Yosys
- 入力: [RTL](https://en.wikipedia.org/wiki/Register-transfer_level) (Verilog)
- 出力: 素子情報付きの[ネットリスト](https://en.wikipedia.org/wiki/Netlist) (Verilog)
- マインメトリック: セルの数

In [ ]:
#@title 📦 Yosysのインストール {display-mode: "form"}
#@markdown - ▷ ボタンをクリックすると、Yosysのセットアップが開始されます。

from IPython.core import magic_arguments
from IPython.core.magic import register_cell_magic
from google.colab import widgets
import pandas as pd
import graphviz

import pathlib
import json

yosys_version = '2024-11-28' #@param {type:"string"}
yosys_version_1 = yosys_version.replace('-', '')
print(f'📦 downloading yosys-{yosys_version}')
!test -d oss-cad-suite || (curl --show-error -L  https://github.com/YosysHQ/oss-cad-suite-build/releases/download/{yosys_version}/oss-cad-suite-linux-x64-{yosys_version_1}.tgz | tar xzf -)

yosys_path = 'oss-cad-suite/bin/yosys'
work_dir = pathlib.Path('xls_work_dir')

def yosys(line: str, cell: str):
  tb = widgets.TabBar(['logs', 'netlist', 'graph', 'metrics'])
  yosys_script = work_dir / 'yosys.script'
  user_module_synth_v = work_dir / 'user_module_synth.v'
  synth_metrics_json = work_dir / 'synth_metrics.json'
  synth_graph_dot = work_dir / 'synth_graph.dot'
  with yosys_script.open('w') as f:
    f.write(cell)
  with tb.output_to('logs', select=True):
    !{yosys_path} -s '{yosys_script}'
  with tb.output_to('netlist', select=True):
    with user_module_synth_v.open() as f:
      print(f.read())
  with tb.output_to('graph', select=True):
    display(graphviz.Source.from_file(synth_graph_dot))
  with tb.output_to('metrics', select=True):
    with synth_metrics_json.open() as f:
      metrics = json.load(f)
      display(
          pd.DataFrame.from_records([metrics['design']])
          .transpose()
          .set_axis(['metrics'], axis=1)
      )

register_cell_magic(yosys)

iverilog_path = 'oss-cad-suite/bin/iverilog'
!{sys.executable} -m pip install https://github.com/Toroid-io/vcd2wavedrom/archive/refs/heads/main.zip wavedrom vcdvcd
import wavedrom
from vcd2wavedrom import vcd2wavedrom

def iverilog(line: str, cell: str):
  user_module_sv = work_dir / 'user_module.sv'
  user_module_vcd = pathlib.Path('user_module.vcd')
  user_module_tb_sv = work_dir / 'user_module_tb.sv'
  with user_module_tb_sv.open('w') as f:
    f.write(cell)
  iverilog_output=!{iverilog_path} -g2012 {user_module_tb_sv} {user_module_sv}
  vvp_output=!./a.out
  with user_module_vcd.open('r') as f:
    vcd = vcd2wavedrom.VCD2Wavedrom({'input_text': f.read()})
    drom = vcd.execute(True)
    svg = wavedrom.render(json.dumps(drom))
    display(svg)

register_cell_magic(iverilog)

print(f'✅ installed yosys-{yosys_version}')

In [ ]:
%%yosys

read_verilog -sv xls_work_dir/user_module.sv
hierarchy -top user_module
proc
flatten

synth
abc -liberty org_theopenroadproject_asap7sc7p5t_28/asap7_rvt_1x_SS.lib
dfflibmap -liberty org_theopenroadproject_asap7sc7p5t_28/asap7_rvt_1x_SS.lib

opt_expr
opt_clean

write_verilog xls_work_dir/user_module_synth.v

show -format dot -prefix xls_work_dir/synth_graph
tee -q -o xls_work_dir/synth_metrics.json stat -liberty org_theopenroadproject_asap7sc7p5t_28/asap7_rvt_1x_SS.lib -json

## 🛣️ Place & Route

### フロアプラン

- 入力: 素子情報付きの[ネットリスト](https://en.wikipedia.org/wiki/Netlist) (Verilog)
- 出力: 電源供給配線網(Power Delivery Network: PDN)とI/Oピンのついたダイ上のレイアウト([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- メトリック: コアの面積

### 配置

- 入力: 素子情報付きの[ネットリスト](https://en.wikipedia.org/wiki/Netlist) (Verilog),
PDNとI/Oピンつきのダイの物理レイアウト([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- 出力: コンポーネントのセルが配置された物理レイアウト ([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- メトリック: コア面積, セル密度, [タイミング収束](https://en.wikipedia.org/wiki/Timing_closure)の推定値

### 配線

- 入力: 素子情報付きの[ネットリスト](https://en.wikipedia.org/wiki/Netlist) (Verilog),
コンポーネントのセルが配置された物理レイアウト([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- 出力: コンポーネントのセルへの配線が完了した物理レイアウト ([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- メトリック: ルートの衝突, [タイミング収束](https://en.wikipedia.org/wiki/Timing_closure)の予測値

In [ ]:
#@title 📦 OpenROADのインストール {display-mode: "form"}
#@markdown - ▷ ボタンをクリックすると、OpenROADのセットアップが開始されます。

import IPython.display
import PIL.Image

import pandas as pd

openroad_version = '2.0-17198-g8396d0866' #@param {type:"string"}

print(f'📦 downloading openroad-{openroad_version}')
!which openroad || (curl -O --show-error -L https://github.com/Precision-Innovations/OpenROAD/releases/download/{openroad_version}/openroad_{openroad_version}_amd64-ubuntu-22.04.deb && apt -qq install ./openroad_{openroad_version}_amd64-ubuntu-22.04.deb)

work_dir = pathlib.Path('xls_work_dir')

def openroad(line: str, cell: str):
  tb = widgets.TabBar(['logs', 'floorplan', 'placement', 'routing', 'metrics'])
  openroad_tcl = work_dir / 'openroad.tcl'
  openroad_metrics_json = work_dir / 'openroad_metrics.json'
  layout_floorplan_png = work_dir / 'layout_floorplan.png'
  layout_placement_png = work_dir / 'layout_placement.png'
  layout_routing_png = work_dir / 'layout_routing.png'
  with openroad_tcl.open('w') as f:
    f.write(cell)
  with tb.output_to('logs', select=True):
    !QT_QPA_PLATFORM=minimal openroad -metrics {openroad_metrics_json} -exit {openroad_tcl}
  with tb.output_to('floorplan', select=True):
    img = PIL.Image.open(layout_floorplan_png)
    img = img.resize((500, 500))
    IPython.display.display_png(img)
  with tb.output_to('placement', select=True):
    img = PIL.Image.open(layout_placement_png)
    img = img.resize((500, 500))
    IPython.display.display_png(img)
  with tb.output_to('routing', select=True):
    img = PIL.Image.open(layout_routing_png)
    img = img.resize((500, 500))
    IPython.display.display_png(img)
  with tb.output_to('metrics', select=True):
    with openroad_metrics_json.open() as f:
      metrics = json.load(f)
      display(
          pd.DataFrame.from_records([metrics])
          .transpose()
          .set_axis(['metrics'], axis=1)
      )

register_cell_magic(openroad)

print(f'✅ installed openroad-{openroad_version}')

In [ ]:
%%openroad

# フロアプラン
read_lef asap7/../org_theopenroadproject_asap7sc7p5t_28/techlef_misc/asap7_tech_1x_201209.lef
read_lef asap7/../org_theopenroadproject_asap7sc7p5t_28/LEF/asap7sc7p5t_28_R_1x_220121a.lef
read_liberty asap7/../org_theopenroadproject_asap7sc7p5t_28/asap7_rvt_1x_SS.lib
read_verilog xls_work_dir/user_module_synth.v
link_design user_module
initialize_floorplan -site "asap7sc7p5t"  -die_area "0 0 8 10" -core_area "1 1 7 9"
source asap7/dependency_support/org_theopenroadproject_asap7_pdk_r1p7/tracks.tcl
insert_tiecells TIEHIx1_ASAP7_75t_R/H -prefix "TIE_ONE_"
insert_tiecells TIELOx1_ASAP7_75t_R/L -prefix "TIE_ZERO_"
create_clock [get_ports clk] -period 0.8
source asap7/dependency_support/org_theopenroadproject_asap7_pdk_r1p7/rc_script.tcl
set_wire_rc -signal -layer "M2"
set_wire_rc -clock  -layer "M5"
place_pins -hor_layers M4 -ver_layers M5
tapcell -distance 25 -tapcell_master TAPCELL_ASAP7_75t_R
source asap7/dependency_support/org_theopenroadproject_asap7_pdk_r1p7/pdn_config.pdn
pdngen -verbose

save_image -resolution 0.005 "xls_work_dir/layout_floorplan.png"

# 配置
global_placement -timing_driven -routability_driven -density 0.95 -pad_left 1 -pad_right 1
remove_buffers
estimate_parasitics -placement
repair_design
repair_timing
detailed_placement
improve_placement
optimize_mirroring
check_placement -verbose

save_image -resolution 0.005 "xls_work_dir/layout_placement.png"

# 配線
set_global_routing_layer_adjustment M4 0.5
set_global_routing_layer_adjustment M2 0.5
set_global_routing_layer_adjustment M7 0.5
set_global_routing_layer_adjustment M3 0.5
set_global_routing_layer_adjustment M6 0.5
set_global_routing_layer_adjustment M5 0.5
set_routing_layers -signal M2-M7 -clock M2-M7
global_route -congestion_iterations 100
set_propagated_clock [all_clocks]
detailed_route

save_image -resolution 0.005 "xls_work_dir/layout_routing.png"

report_power
report_design_area
report_wire_length -net * -detailed_route
report_wns
utl::metric "utilization_percent" [rsz::utilization]
utl::metric "design_area" [rsz::design_area]
utl::metric "power" [sta::design_power [sta::parse_corner {}]]
utl::metric "wns" [sta::worst_slack -max]

## 🏋️‍♂️ 演習課題

### ➕ 足し算

1. 下記の１ビットの足し算を試し
1. 回路合成の後：ゲートの種類の理解 ([ヒント](https://ja.wikipedia.org/wiki/%E8%AB%96%E7%90%86%E5%9B%9E%E8%B7%AF#%E7%B5%84%E3%81%BF%E5%90%88%E3%82%8F%E3%81%9B%E5%9B%9E%E8%B7%AF))
1. Place & Routeの後：ゲートのつなぎ方の理解
1. 足し算のビット数を増やすこと ([ヒント](https://google.github.io/xls/dslx_reference/#bit-type))



In [ ]:
%%dslx --top=adder1 --pipeline_stages=1 --flop_inputs=false --flop_outputs=false

fn adder1(a: u1, b: u1) -> u2 {
    a as u2 + b as u2
}

#[test]
fn adder1_test() {
  assert_eq(adder1(u1:0, u1:0), u2:0b00);
  assert_eq(adder1(u1:0, u1:1), u2:0b01);
  assert_eq(adder1(u1:1, u1:0), u2:0b01);
  assert_eq(adder1(u1:1, u1:1), u2:0b10);
}

### ✖️ 掛け算

1. 下記の8ビットの掛け算の設計
1. 回路合成の後：ゲートの数の理解
1. scheduleのタイミングをの確認
1. stdライブラリを使っても16ビットのoutputの変更

    ヒント：
    ```
    import std;
    
    fn mul8(a: u8, b: u8) -> u16 {
      // ...
    }
    ```
    [std::umul](https://google.github.io/xls/dslx_std/#stdmul)

1. scheduleのタイミングをの確認


In [ ]:
%%dslx --top=mul8 --pipeline_stages=1 --flop_inputs=false --flop_outputs=false

import std;

fn mul8(a: u8, b: u8) -> u8 {
    a * b
}

#[test]
fn mul8_test() {
  assert_eq(mul8(u8:2, u8:2), u8:4);
  assert_eq(mul8(u8:128, u8:2), u8:0);
}

### 👷 パイプライン

1. 足し算と掛け算の回路の組み合わせ
1. scheduleのタイミングをの確認
1. clock_period_psを800psに下げること、scheduleのステージの数の確認

    ヒント：[命令パイプライン](https://ja.wikipedia.org/wiki/%E5%91%BD%E4%BB%A4%E3%83%91%E3%82%A4%E3%83%97%E3%83%A9%E3%82%A4%E3%83%B3)


In [ ]:
%%dslx --top=muladd --clock_period_ps=800 --flop_inputs=true --flop_outputs=true
                    // TODO: clock_period_psの設定をいじってみてください

import std;

fn muladd(a: u8, b: u8, c: u8) -> u16 {
    // TODO：ここに「a」と「b」の掛け算と「c」の足し算を書き直して下さい
    u16:0
}

#[test]
fn muladd_test() {
  assert_eq(muladd(u8:2, u8:2, u8:2), u16:6);
  assert_eq(muladd(u8:16, u8:16, u8:16), u16:272);
  // TODO：ここにtestsを増やして下さい。
}

### 🧪 シミュレーション

1. シミュレーションを動かすこと。
1. テストインプットの変更。
1. `clk`と他の信号の動きの理解（前のverilogタブをチェックしながら）


In [ ]:
%%iverilog

module user_module_tb;
  reg clk;
  wire [15:0] out;
  // TODO:`2`の変わり他のテストインプットを選んで下さい（`8`はビット数、`h`はHEX、`d`はDEC）。
  user_module dut(clk, 8'h2, 8'h2, 8'h2, out);
always #1 clk = ~clk;
initial begin
  clk = 0;
  $dumpfile("user_module.vcd");
  $dumpvars(0, dut);
  #10 $finish;
end
endmodule

## 🏆 チャレンジ課題

1. DSLXもっと詳しく学びたい人 → https://bit.ly/learn-xls
1. 浮動小数点数の足し算の設計 → https://google.github.io/xls/floating_point/
1. 自分のチップを作りたい人 → https://tinytapeout.com/